In [1]:
import os
import sys
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = '1, 2'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
dirs = ["/home/leisaihua/workspace/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType, CustomTemplateType

# 官方方法选择模型
# model_type = ModelType.mistral_7b_instruct_v3
# model_type = ModelType.llama3_70b_instruct_awq
# 从custom.py中设定模型
# model_type = ModelType.llama3_8b_instruct
# model_type = ModelType.llama3_1_8b_instruct
# model_type = CustomModelType.mixtral_moe_7b_instruct_awq
model_type = CustomModelType.solar_instruct_10_7b


llm_engine = get_vllm_engine(
    model_type, 
    # 使用官方方法，需要指定模型路径，否则会自己下载模型
    model_id_or_path="/data/leisaihua/models/SOLAR-10.7B-Instruct-v1.0",
    # model_id_or_path="/home/css/models/Mistral-7B-Instruct-v0.3",
    tensor_parallel_size=2,
    max_model_len=4096,
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 64,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

/home/leisaihua/miniconda3/envs/swift/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/leisaihua/miniconda3/envs/swift/lib/python3.10/site-packages/awq/modules/linear/exllama.py:12: UserWarning: AutoAWQ could not load ExLlama kernels extension. Details: /home/leisaihua/miniconda3/envs/swift/lib/python3.10/site-packages/exl_ext.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN3c104cuda9SetDeviceEi
  warnings.warn(f"AutoAWQ could not load ExLlama kernels extension. Details: {ex}")
/home/leisaihua/miniconda3/envs/swift/lib/python3.10/site-packages/awq/modules/linear/exllamav2.py:13: UserWarning: AutoAWQ could not load ExLlamaV2 kernels extension. Details: /home/leisaihua/miniconda3/envs/swift/lib/python3.10/site-packages/exlv2_ext.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN3c104cuda9Se

INFO 08-14 20:04:15 config.py:729] Defaulting to use mp for distributed inference
INFO 08-14 20:04:16 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/data/leisaihua/models/SOLAR-10.7B-Instruct-v1.0', speculative_config=None, tokenizer='/data/leisaihua/models/SOLAR-10.7B-Instruct-v1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=42, served_model_name=/data/leisaihua/models/SOLAR-10.7B-Instruct-v1.0, use_v2_block_manager=False, enable_prefix_c

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:16<01:06, 16.71s/it]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:37<00:58, 19.38s/it]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:55<00:36, 18.42s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [01:03<00:14, 14.42s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [01:34<00:00, 20.27s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [01:34<00:00, 18.84s/it]



INFO 08-14 20:05:55 model_runner.py:732] Loading model weights took 9.9978 GB
(VllmWorkerProcess pid=2837378) INFO 08-14 20:05:55 model_runner.py:732] Loading model weights took 9.9978 GB
INFO 08-14 20:05:57 distributed_gpu_executor.py:56] # GPU blocks: 7281, # CPU blocks: 2730
INFO 08-14 20:05:59 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-14 20:05:59 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=2837378) INFO 08-14 20:05:59 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in ea

In [2]:
from utils import tools
import json


filepath = 'utils/data/LE-dataset v2.1/llama3-newdata/LE-dataset v2.1.2-150.json'

request_list = []
latin_phrase = []
local_sentence = []
with open(filepath, 'r') as file:
    contents = json.load(file)
    for i, ele in enumerate(contents):
        # if i > 100:
        #     break
        phrase = ele['Latin phrase']
        text = ele['llmed sentence']
        eval_prompt = f"Replace the Latin phrase in the sentence with equivalent English content without changing the original meaning. Just tell me the answer, no explanation. You should respond in the following format: A: Replaced content is <###>, Replaced sentence is <###>.\nExamples\nQ: Latin phrase is <de facto>, Sentence is <Previously, Disney had de facto control of the board as the district's largest landowner.>\nA: Replaced content is <actual>, Replaced sentence is <Previously, Disney had actually control of the board as the district's largest landowner.>\nQ: Latin phrase is <sub rosa>, Sentence is <The idea that that energy is continuing 25 years later, in a kind of sub rosa away, was very fun for us.>\nA: Replaced content is <secret>, Replaced sentence is <The idea that that energy is continuing 25 years later, in a kind of secret way, was very fun for us.>\nQuestion:\nQ: Latin phrase is <{phrase}>, Sentence is <{text}>\n"
        # 获取模型的回答
        request_list.append({'query': eval_prompt})
        latin_phrase.append(phrase)
        local_sentence.append(text)
print(len(request_list), request_list[0])


# content_list = []

# print("需要生成的语句数量：", len(content_list))
# # 请求内容
# request_list = []
# latin_phrase = []
# local_sentence = []
# for i in range(0, len(content_list)):
#     content_tuple = content_list[i]
#     phrase, text = content_tuple
#     phrase = str(phrase)
#     text = str(text)
#     # 设计提示词
#     eval_prompt = f"Replace the Latin phrase in the sentence with equivalent English content without changing the original meaning. Just tell me the answer, no explanation. You should respond in the following format: A: Replaced content is <###>, Replaced sentence is <###>.\nExamples\nQ: Latin phrase is <de facto>, Sentence is <Previously, Disney had de facto control of the board as the district's largest landowner.>\nA: Replaced content is <actual>, Replaced sentence is <Previously, Disney had actually control of the board as the district's largest landowner.>\nQ: Latin phrase is <sub rosa>, Sentence is <The idea that that energy is continuing 25 years later, in a kind of sub rosa away, was very fun for us.>\nA: Replaced content is <secret>, Replaced sentence is <The idea that that energy is continuing 25 years later, in a kind of secret way, was very fun for us.>\nQuestion:\nQ: Latin phrase is <{phrase}>, Sentence is <{text}>\n"
    
#     # eval_prompt = f"Replace the Latin phrase in the sentence with equivalent English content without changing the original meaning. Just tell me the answer, no explanation. You should respond in the following format: A: Replaced content is <<<###>>>, Replaced sentence is <<<###>>>.\nExamples\nQ: Latin phrase is <<<de facto>>>, Sentence is <<<Previously, Disney had de facto control of the board as the district's largest landowner.>>>\nA: Replaced content is <<<actual>>>, Replaced sentence is <<<Previously, Disney had actually control of the board as the district's largest landowner.>>>\nQ: Latin phrase is <<<sub rosa>>>, Sentence is <<<The idea that that energy is continuing 25 years later, in a kind of sub rosa away, was very fun for us.>>>\nA: Replaced content is <<<secret>>>, Replaced sentence is <<<The idea that that energy is continuing 25 years later, in a kind of secret way, was very fun for us.>>>\nQuestion:\nQ: Latin phrase is <<<{phrase}>>>, Sentence is <<<{text}>>>\n"
        
#     # 获取模型的回答
#     request_list.append({'query': eval_prompt})
#     latin_phrase.append(phrase)
#     local_sentence.append(text)

64847 {'query': "Replace the Latin phrase in the sentence with equivalent English content without changing the original meaning. Just tell me the answer, no explanation. You should respond in the following format: A: Replaced content is <###>, Replaced sentence is <###>.\nExamples\nQ: Latin phrase is <de facto>, Sentence is <Previously, Disney had de facto control of the board as the district's largest landowner.>\nA: Replaced content is <actual>, Replaced sentence is <Previously, Disney had actually control of the board as the district's largest landowner.>\nQ: Latin phrase is <sub rosa>, Sentence is <The idea that that energy is continuing 25 years later, in a kind of sub rosa away, was very fun for us.>\nA: Replaced content is <secret>, Replaced sentence is <The idea that that energy is continuing 25 years later, in a kind of secret way, was very fun for us.>\nQuestion:\nQ: Latin phrase is <pros and cons>, Sentence is <Investing in commercial real estate comes with pros and cons.>\n

In [3]:
generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

# 请求内容格式
# request_list = [
#     {'query': "Hi?"},
# ]

# 回答内容
resp_list = inference_vllm(
    llm_engine, template, request_list, generation_config=generation_config, 
    # 显示进度条
    use_tqdm=True
)

100%|██████████| 64847/64847 [1:19:04<00:00, 13.67it/s]


In [4]:
# 保存内容
response_list = []
for i, (phrase, text, request, resp) in enumerate(zip(latin_phrase, local_sentence, request_list, resp_list)):
    # print(i+1, '------------------------------------')
    # print(f"query:\n{request['query']}")
    # print(f"response:\n{resp['response']}")
    # print(i+1, resp['response'])
    response = str(resp['response'])    
    
    # llmed response: 大模型的回答
    response_list.append({'Latin phrase': phrase, 'local sentence': text, 'llmed response': response})
print(len(response_list))
# 指定本地文件路径
out_filepath = './utils/data/LE-dataset v2.1/llm-output v2.1/solar-out/LE-dataset v2.1.2-150.json'
tools.save_response(out_filepath, response_list)

64847


#### 使用LLMs生成新的数据集

In [18]:
from utils import tools


filepath = './utils/data/LE-dataset v2.1/LE-dataset v2.1-150.json'
# 生成用于微调的数据集 892条
content_list = tools.get_LE_data(filepath)
print("需要生成的语句数量：", len(content_list))

# 请求内容
request_list = []
latin_phrase = []
local_sentence = []
for i in range(0, len(content_list)):
    content_tuple = content_list[i]
    phrase, text = content_tuple
    phrase = str(phrase)
    text = str(text)
    # 设计提示词
    # eval_prompt = f"Replace the Latin phrase in the sentence with equivalent English content without changing the original meaning. Just tell me the answer, no explanation. You should respond in the following format: A: Replaced content is <###>, Replaced sentence is <###>.\nExamples\nQ: Latin phrase is <de facto>, Sentence is <Previously, Disney had de facto control of the board as the district's largest landowner.>\nA: Replaced content is <actual>, Replaced sentence is <Previously, Disney had actually control of the board as the district's largest landowner.>\nQ: Latin phrase is <sub rosa>, Sentence is <The idea that that energy is continuing 25 years later, in a kind of sub rosa away, was very fun for us.>\nA: Replaced content is <secret>, Replaced sentence is <The idea that that energy is continuing 25 years later, in a kind of secret way, was very fun for us.>\nQuestion:\nQ: Latin phrase is <{phrase}>, Sentence is <{text}>\n"
    eval_prompt = f"Extract the following sentences that contain Latin phrase. Just tell the extracted sentence.\nQ: Latin phrase is <jus ad bellu>, the contence is <'g. the jus ad bellum) are, arguably, one of the greatest obstacles with regard to achieving a lasting Asian and European peace. A few examples should amply demonstrate why.'>.\nA: Extracted sentence is <The jus ad bellum are, arguably, one of the greatest obstacles with regard to achieving a lasting Asian and European peace.>.\nQ: Latin phrase is <ex vivo>, the sentnece is <E. Limitations of ex vivo measurements for in vivo neuroscience. Proc.>.\nA: Extracted sentence is <Limitations of ex vivo measurements for in vivo neuroscience.>.\nQ: Latin phrase is <{phrase}>, the sentnece is <{text}>.\n"
    
    # eval_prompt = f"Replace the Latin phrase in the sentence with equivalent English content without changing the original meaning. Just tell me the answer, no explanation. You should respond in the following format: A: Replaced content is <<<###>>>, Replaced sentence is <<<###>>>.\nExamples\nQ: Latin phrase is <<<de facto>>>, Sentence is <<<Previously, Disney had de facto control of the board as the district's largest landowner.>>>\nA: Replaced content is <<<actual>>>, Replaced sentence is <<<Previously, Disney had actually control of the board as the district's largest landowner.>>>\nQ: Latin phrase is <<<sub rosa>>>, Sentence is <<<The idea that that energy is continuing 25 years later, in a kind of sub rosa away, was very fun for us.>>>\nA: Replaced content is <<<secret>>>, Replaced sentence is <<<The idea that that energy is continuing 25 years later, in a kind of secret way, was very fun for us.>>>\nQuestion:\nQ: Latin phrase is <<<{phrase}>>>, Sentence is <<<{text}>>>\n"
        
    # 获取模型的回答
    request_list.append({'query': eval_prompt})
    latin_phrase.append(phrase)
    local_sentence.append(text)

需要生成的语句数量： 66627


In [19]:
generation_config = VllmGenerationConfig(
    max_new_tokens=4096,
    temperature=0,
)

# 请求内容格式
# request_list = [
#     {'query': "Hi?"},
# ]

# 回答内容
resp_list = inference_vllm(
    llm_engine, template, request_list, generation_config=generation_config, 
    # 显示进度条
    use_tqdm=True
)

  0%|          | 127/66627 [00:31<4:06:08,  4.50it/s]

WARNING 08-11 13:13:51 scheduler.py:1099] Sequence group 163 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


  3%|▎         | 1814/66627 [06:57<4:03:52,  4.43it/s]

WARNING 08-11 13:20:17 scheduler.py:1099] Sequence group 1846 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=101


  5%|▍         | 3160/66627 [12:37<6:10:50,  2.85it/s]

...

 96%|█████████▌| 63683/66627 [4:08:29<11:17,  4.35it/s]

WARNING 08-11 17:21:49 scheduler.py:1099] Sequence group 63722 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1951


 99%|█████████▊| 65659/66627 [4:16:12<04:18,  3.75it/s]

WARNING 08-11 17:29:32 scheduler.py:1099] Sequence group 65694 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=2001


100%|██████████| 66627/66627 [4:19:59<00:00,  4.27it/s]


In [30]:
# 保存内容
response_list = []
for i, (phrase, text, request, resp) in enumerate(zip(latin_phrase, local_sentence, request_list, resp_list)):
    # print(i+1, '------------------------------------')
    # print(f"query:\n{request['query']}")
    # print(f"response:\n{resp['response']}")
    # print(i+1, resp['response'])
    response = str(resp['response'])    
    import re
    
    pattern = r"<(.*?)>"
    
    matches = list(re.finditer(pattern, response, flags=re.DOTALL))
    # print(len(matches))
    if len(list(matches)) == 1:
        for match in matches:
            replaced_content = match.group(1)
            # print(replaced_content)
            if phrase not in replaced_content:
                continue
            
            # llmed response: 大模型的回答
            response_list.append({'Latin phrase': phrase, 'local sentence': text, 'llmed response': response, 'llmed sentence': replaced_content})
print(len(response_list))
# 指定本地文件路径
out_filepath = './utils/data/LE-dataset v2.1/llama3-newdata/LE-dataset v2.1.2-150.json'
tools.save_response(out_filepath, response_list)

64847
